In [166]:
import numpy as np
import collections

In [167]:
def read(dataset_year: str, max_docs_count:int = None):
  is_text = False
  docs_count = 0

  with open(f'dataset/cg.Folha.{dataset_year}', encoding='windows-1252') as file: 
    for line in file:
      if line.startswith('<DOC>'):
        docs_count += 1
      
      if docs_count > max_docs_count:
        break

      if line.startswith('<s>'):
        is_text = True
        continue
      if line.startswith('</s>'):
        is_text = False
        continue

      splitted_line = line.split('\t')
      is_valid_line = len(splitted_line) == 2

      if is_text and is_valid_line: 
        yield (splitted_line[0], splitted_line[1])

In [168]:
def parse(word: str, word_info: str):  
  tag = word_info.split()[1]
  
  return (word, tag)


In [169]:
parsed_words = [parse(word, word_info) for word, word_info in read('1994', max_docs_count=1)]

word_tag_counter = collections.Counter(parsed_words)
tag_counter = collections.Counter([tag for _, tag in word_tag_counter])
tag_counter

Counter({'PROP': 18,
         'V': 7,
         'PRP': 5,
         'DET': 5,
         'NUM': 7,
         'ADJ': 7,
         'N': 13,
         'PU': 5,
         'ADV': 2,
         'KC': 1,
         '<prop>': 1})